<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>
<center> <font size ="6" color='red'> Production Planning Cockpit </font></center>


In [1]:
from DBMain import*
#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); 
tab_5 = VisMgr.getProductionProgressTab().generatePPrgTAB()
tab_3 = VisMgr.generatePLTAB(); tab_6 = VisMgr.getSchedulingTab().generatePSschTAB()
tab_7 = VisMgr.getSimulationTab().generateSimTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_5,tab_3,tab_6,tab_7])

tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Instance Data');tab_set.set_title(2, 'Production Progress')
tab_set.set_title(3, 'Planning');tab_set.set_title(4,'Scheduling');tab_set.set_title(5, 'Simulation'); tab_set

In [7]:
a = DataMgr

In [8]:
a.getResources()

{'M3-02 - Accuway - 3axis - Manual (FR3_02)': <PlanningObjects.Resource at 0x14c57c66510>,
 'M5-04 - Haas - 5axis  - Manual (UMC750SS)': <PlanningObjects.Resource at 0x14c57a9f6e0>,
 'MP1 - Material preparation - automatic sawing (ZGN_01)': <PlanningObjects.Resource at 0x14c57b71df0>,
 'M5-02 - Accuway - 5axis - Pallet Automation (FR5_02)': <PlanningObjects.Resource at 0x14c57aa4c80>,
 'M5-01 - Accuway - 5axis - Conveyor Automation (FR5_01)': <PlanningObjects.Resource at 0x14c57a9e780>,
 'BW - Benchwork (BKW_01)': <PlanningObjects.Resource at 0x14c57aa6540>,
 'PACK - Packaging (VERP_P)': <PlanningObjects.Resource at 0x14c57aa6480>,
 'M3-01 - Accuway - 3axis - Manual (FR3_01)': <PlanningObjects.Resource at 0x14c57c65040>,
 'M4-02 - Accuway - 4axis - Conveyor Automation (FR4_02)': <PlanningObjects.Resource at 0x14c57c66780>,
 'OUT - Outsourced activity': <PlanningObjects.Resource at 0x14c57c66630>,
 'MP3 - Material preparation - pre-cutted, only picking (UITG_M)': <PlanningObjects.Resour

In [13]:
a.getCustomerOrders()['[7005-2480-0090]  METROFRAME BODY V02-00-B / 0119073001 _4.0[92e1]']

In [18]:
test = []

In [23]:
for key, order in a.getCustomerOrders().items():
    if len(order.getMyJobs()) > 0:
        test.append(order.getMyJobs())

AttributeError: 'list' object has no attribute 'getActualStart'

In [20]:
test[0]

In [21]:
b = test[0]

In [26]:
for i in b:
    print(i.getMySch().getScheduledCompletion())
        

2026-01-08 16:30:00
2026-01-09 12:30:00
2026-01-12 13:30:00
2026-01-12 14:00:00
2026-01-12 14:30:00


In [27]:
sched = SchedulingMgr

In [32]:
print(sched.getMyCurrentSchedule().getResourceSchedules())

{'M3-02 - Accuway - 3axis - Manual (FR3_02)': {<PlanningObjects.Shift object at 0x0000014C57BA0CE0>: {}, <PlanningObjects.Shift object at 0x0000014C566377D0>: {}, <PlanningObjects.Shift object at 0x0000014C57C672F0>: {}, <PlanningObjects.Shift object at 0x0000014C57C67290>: {}, <PlanningObjects.Shift object at 0x0000014C57C675C0>: {}, <PlanningObjects.Shift object at 0x0000014C57C67230>: {}, <PlanningObjects.Shift object at 0x0000014C57C67140>: {}, <PlanningObjects.Shift object at 0x0000014C57C674D0>: {}, <PlanningObjects.Shift object at 0x0000014C57C66570>: {}, <PlanningObjects.Shift object at 0x0000014C57C67590>: {}, <PlanningObjects.Shift object at 0x0000014C57C676B0>: {}, <PlanningObjects.Shift object at 0x0000014C57C67680>: {}, <PlanningObjects.Shift object at 0x0000014C57C66900>: {}, <PlanningObjects.Shift object at 0x0000014C57C66990>: {}, <PlanningObjects.Shift object at 0x0000014C57C66DB0>: {}, <PlanningObjects.Shift object at 0x0000014C57C66AE0>: {}, <PlanningObjects.Shift ob

In [39]:
sched.getMyShifts()[datetime.date(2026, 1, 8)][1].getStartTime()

32